In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import DataLoader

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,groups=2),nn.RuLU(inplace=True))
        self.norm1=nn.LocalResponseNorm(size=5,alpha=1e-4,beta=0.75,k=2)
        self.pool1=nn.MaxPool2d(kernel_size=3,stride=2)

        self.conv2=nn.Sequential(nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,stride=4,groups=2),nn.RuLU(inplace=True))
        self.norm2=nn.LocalResponseNorm(size=5,alpha=1e-4,beta=0.75,k=2)
        self.pool2=nn.MaxPool2d(kernel_size=3,stride=2)


        self.conv3=nn.Sequential(nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3),nn.RuLU(inplace=True))
        self.conv4=nn.Sequential(nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,groups=2),nn.RuLU(inplace=True))
        self.conv5=nn.Sequential(nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,groups=2),nn.RuLU(inplace=True))
        self.pool3=nn.MaxPool2d(kernel_size=3,stride=2)

        self.fc1=nn.Sequential(nn.Linear(256*6*6,4096),nn.ReLU(inplace=True))
        self.fc2=nn.Sequential(nn.Linear(4096,4096),nn.ReLU(inplace=True))
        self.fc3=nn.Sequential(nn.Linear(4096,1000))

        self.softmax=nn.Softmax(dim=-1)


    def forward(self,x):
        x=self.conv1(x)
        x=self.norm1(x)
        x=self.pool1(x)

        x=self.conv2(x)
        x=self.norm2(x)
        x=self.pool2(x)

        x=self.conv3(x)
        x=self.conv4(x)

        x=self.conv5(x)
        x=self.pool3(x)

        x = torch.flatten(x, start_dim=1)

        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)

        x=self.softmax(x)
        return x



In [ ]:
transform=transforms.ToTensor()
data_root='/content/drive/MyDrive/Colab Notebooks/Data/ILSVRC2012_devkit_t12'
os.chdir(data_root)
train=datasets.ImageNet(root='./data',split='train',transform=transform)
test=datasets.ImageNet(root='./data',split='val',transform=transform)
train_dataloader=DataLoader(train,
            batch_size=4,
            shuffle=True,)
test_dataloader=DataLoader(test,
            batch_size=4,
            shuffle=True)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=AlexNet().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
loss=torch.nn.CrossEntropyLoss()

loss_score=0
best_loss = 100
for epoch in range(200):
    for i,(data,label) in enumerate(train_dataloader):
        data=data.to(device)
        label=label.to(device)
        optimizer.zero_grad()
        output=model(data)
        loss_val=loss(output,label)
        loss_val.backward()
        optimizer.step()

        if i%1000==0:
            print('Epoch:',epoch,'Step:',i,'Loss:',loss_val.item())

    model.eval()
    eval_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # 평가에서는 그래디언트 계산을 하지 않음
        for data, labels in test_dataloader:
            data=data.to(device)
            labels=labels.to(device)
            outputs = model(data)
            loss_test = loss(outputs, labels)
            eval_loss += loss_test.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy:',correct/len(test))
        test_loss=eval_loss/len(test_dataloader)
        print('Test Loss:',test_loss)
        if best_loss>test_loss:
          best_loss=test_loss
          torch.save(model.state_dict(), './alexnet_model_best.pth')